In [22]:
%load_ext autoreload
%autoreload 2
!export CUDA_VISIBLE_DEVICES=5

import sys
    
import re
import os
import json
import torch
import numpy as np

sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [166]:
#!/usr/bin/env python
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage
from langserve import add_routes
from langchain_community.embeddings import HuggingFaceEmbeddings

In [239]:
import torch
from dataclasses import asdict
from src.autodesk_chat.arguments import EmbeddingModelArguments, PageLoaderArguments, TextSplitterArguments
from src.autodesk_chat.page_loaders import PageLoader
from src.autodesk_chat.text_splitters import RecursiveCharacterTextSplitterAndFilterer, HTMLHeaderTextSplitterAndFilterer, HTMLHeaderTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from transformers import HfArgumentParser

In [221]:
config_file_path = "../helpers/configs/create_vector_db_configs.json"

In [222]:
parser = HfArgumentParser([PageLoaderArguments, EmbeddingModelArguments, TextSplitterArguments])
page_loader_args, embed_model_args, text_splitter_args = parser.parse_json_file(config_file_path)
page_loader_args: PageLoaderArguments
embed_model_args: EmbeddingModelArguments
text_splitter_args: TextSplitterArguments
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embed_model_args.model_kwargs['device'] = device
page_loader_args.pages_dir = "../data/pages/"
text_splitter_args

TextSplitterArguments(splitter_type='html_splitter', min_words_drop=2, parent_text_splitter_kwargs={'headers_to_split_on': [['h1', 'Header 1'], ['h2', 'Header 2'], ['h3', 'Header 3']]})

In [163]:
# Loading the page loader
page_loader = PageLoader(**asdict(page_loader_args))

In [82]:
# Loading the embedding model
hf_embedding = HuggingFaceEmbeddings(**asdict(embed_model_args))

In [83]:
docs = page_loader.load()

In [156]:
raw_pages = list(page_loader.load_raw_pages())

In [223]:
text_splitter = HTMLHeaderTextSplitterAndFilterer(
    min_words_drop=text_splitter_args.min_words_drop,
    **text_splitter_args.parent_text_splitter_kwargs
)
# documents = text_splitter.split_documents(docs)
# documents = text_splitter.split_text()

In [243]:
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=[("h1", "head1"), ("h2", "head2"), ("h3", "head3")])

In [268]:
# docs_list_hmtl_split = html_splitter.split_text(raw_pages[0].page_content)
rec_splitter = RecursiveCharacterTextSplitterAndFilterer(min_words_drop=7, do_clean=True, chunk_size=1000)
# RecursiveCharacterTextSplitter(separators=)
documents_from_rec_spt = rec_splitter.split_documents(docs)
# rec_splitterdocs_list_hmtl_split[0]
# raw_pages[0].

In [269]:
len(documents_from_rec_spt)

10985

In [270]:
len(documents_from_rec_spt)

10985

In [271]:
documents_from_rec_spt[0]

Document(page_content='Advance Steel Key Features 2022 | Upcoming Advanced Features 2023 | Autodesk\nContact sales at\nTalk to sales:\nHave Autodesk contact you\nDownload free trial\nSee pricing options\nContact sales at\nTalk to sales:\nHave Autodesk contact you\nDownload free trial\nSee pricing options\nFeatures\nSteel modeling\nDetailing & documentation\nInteroperability\nSystem requirements\nBack\nAdvance Steel features\nAdvance Steel tools drive efficient steel design and workflows, while complementary products in the AEC Collection offer enhanced analytic, automation, and model coordination when working in BIM.\nSteel modeling\nParametric steel connections\nModel more quickly, efficiently, and accurately with a customizable library of ready-to-use steel connections.\nLearn more\nStairs and cage ladders\nDiscover and model non-structural building elements quickly and accurately.\nLearn more\nSheet metal and folded plate work\nCreate 3D folded elements of any sheet metal shape fast

In [229]:
documents = text_splitter.split_documents(raw_pages)

Splitting pages with HTML splitter: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1218/1218 [02:08<00:00,  9.46it/s]


In [283]:
# raw_pages[0].page_content
# print(documents[0].page_content)
# len(documents)
no_header_docs = []
with_header_docs = []
for idoc in documents:
    have_header = []
    for header in ["Header 1", "Header 2", "Header 3"]:
        have_header.append(header in idoc.metadata)
    if not any(have_header):
        no_header_docs.append(idoc)
    else:
        with_header_docs.append(idoc)
            

In [284]:
len(no_header_docs), len(documents), len(with_header_docs)

(2256, 4639, 2383)

In [285]:
rec_splittecd_docs = rec_splitter.split_documents(no_header_docs)

In [291]:
rec_splittecd_docs[0], no_header_docs[0]

(Document(page_content='Contact sales at\nTalk to sales:\nHave Autodesk contact you\nDownload free trial\nSee pricing options\nContact sales at\nTalk to sales:\nHave Autodesk contact you\nDownload free trial\nSee pricing options\nFeatures\nSteel modeling\nDetailing & documentation\nInteroperability\nSystem requirements\nBack\nAdvance Steel features\nAdvance Steel tools drive efficient steel design and workflows, while complementary products in the AEC Collection offer enhanced analytic, automation, and model coordination when working in BIM.\nSteel modeling\nParametric steel connections\nModel more quickly, efficiently, and accurately with a customizable library of ready-to-use steel connections.\nLearn more\nStairs and cage ladders\nDiscover and model non-structural building elements quickly and accurately.\nLearn more\nSheet metal and folded plate work\nCreate 3D folded elements of any sheet metal shape faster.\nLearn more\nDynamo Extension for Advance Steel', metadata={'format': 'ht

In [286]:
no_header_docs[0].metadata

{'format': 'html', 'id': 'adsk-0d6daa6f32e84f4dbc90d9a3c3693e32'}

In [293]:
all_documents = rec_splittecd_docs + documents
len(all_documents)

15207

In [294]:
vector = FAISS.from_documents(all_documents, hf_embedding)

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 476/476 [01:28<00:00,  5.35it/s]


In [332]:
query = "What's the difference between AutoCAD and Revit?"
# query = "What does Fusion 360 do?"
query = "What is Revit?"
i = 10
print(vector.similarity_search_with_score(query, k=30)[i][0].page_content)

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 47.68it/s]

Revit LT: 3D modeling tools for design changemakers
Economical BIM software for inspired architecture
Download free trial
Download free trial
See pricing options
Download free trial
See pricing options
Overview
Capabilities
Customer stories
FAQs
Compare
Back
What is Revit LT?
A BIM (Building Information Modeling) solution for the self-starter, Revit LT™️ is a cost-effective project workhorse for the small architecture studio or solo practitioner.
Get all the design to documentation power of Revit at a fraction of the cost.
Win more work where BIM is required.
Accelerate production workflows and standardize deliverables to reduce rework.
See system requirements
Revit LT overview (video: 1:37 min.)
What you can do with Revit LT
Quickly define and document design intent
With tools for sketching, model family creation, scheduling, annotating, and document production, Revit LT drives efficient BIM workflows for architectural design.
See Revit LT features
Grow your design business around BIM

In [334]:
# vector.save_local("../data/vector_db")
loaded_vector = FAISS.load_local("../data/vector_db/", hf_embedding)

In [329]:
loaded_vector

In [454]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

openai_key = ""
os.environ['OPENAI_API_KEY'] = openai_key 

chat = ChatOpenAI(model="gpt-3.5-turbo-1106")

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions JUST based on the below context."
            " YOU SHOULD NOT PROVIDE ANY INFORMATION NOT MENTIONED IN THE CONTEXT. "
            "If it is not possible to answer user's question with the provided context just mention 'I do not have the knowledge to answer the question. Please refer to the customer service'. "
            " \n\n context: {context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [455]:
document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

In [456]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Answer the user's questions JUST based on the below context. YOU SHOULD NOT PROVIDE ANY INFORMATION NOT MENTIONED IN THE CONTEXT. If it is not possible to answer user's question with the provided context just mention 'I do not have the knowledge to answer the question. Please refer to the customer service'.  \n\n context: {context}")), MessagesPlaceholder(variable_name='me

In [457]:
# query = "What's the difference between AutoCAD and Revit?"
query = "What does Fusion 360 do?"
# query = "Who is the president of USA?"
# i = 50
# print(vector.similarity_search_with_score(query, k=50)[i][0].page_content)
# context = vector.similarity_search_with_score(query, k=50)

In [488]:
retriever = vector.as_retriever(k=15)
context = retriever.invoke(query)
len(context)

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 60.89it/s]


4

In [465]:
context = vector.similarity_search(query, k=20)
# context

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 34.22it/s]


In [464]:
from langchain.memory import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message(query)

document_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
        "context": context,
    }
)

'Fusion 360 is a cloud-based 3D modeling, CAD, CAM, CAE, and PCB software platform for product design and manufacturing. It offers integrated capabilities and features for product development, including electronics design, simulation, collaboration, and machining.'

In [567]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain_core.documents import Document
from typing import Dict

In [514]:

def parse_retriever_input(params: Dict):
    return params["messages"][-1].content
    # return params['messages']


In [599]:
def invoke_vector(params: Dict):
    query = params['query']
    k = params['chunk_num']
    return vector.similarity_search(query=query, k=k)

In [516]:
retrieval_chain = RunnablePassthrough.assign(
    context=RunnablePassthrough.assign(query=parse_retriever_input) | RunnableLambda(invoke_vector),
).assign(
    answer=document_chain,
)

In [517]:
input_dict = {
        "messages": demo_ephemeral_chat_history.messages,
        "chunk_num": 15,
        "akbar": 12
    }

In [518]:
# lm = RunnablePassthrough.assign(query=parse_retriever_input) | RunnableLambda(invoke_vector)
get_query = RunnablePassthrough.assign(query=parse_retriever_input)
get_query.invoke(input_dict)
# lm.invoke({
#     "messages": query,
#     "chunk_num": 15
# })
# lm

{'messages': [HumanMessage(content='What does Fusion 360 do?')],
 'chunk_num': 15,
 'akbar': 12,
 'query': 'What does Fusion 360 do?'}

In [521]:
demo_ephemeral_chat_history.add_ai_message(response['answer'])

In [519]:
response = retrieval_chain.invoke(
    input_dict
)

response

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 61.60it/s]


{'messages': [HumanMessage(content='What does Fusion 360 do?')],
 'chunk_num': 15,
 'akbar': 12,
 'context': [Document(page_content='Download Fusion 360 for personal, hobby use. Need full features and functionality?', metadata={'format': 'html', 'id': 'adsk-b6a6cc8e4b924d3897b28275d9107e93', 'Header 2': "Let's get started"}),
  Document(page_content='Download Fusion 360 for personal, hobby use.', metadata={'format': 'html', 'id': 'adsk-d16aba2dadfa44b59e276112652f582e', 'Header 2': "Let's get started"}),
  Document(page_content='Download Fusion 360 for personal, hobby use.', metadata={'format': 'html', 'id': 'adsk-8ad2479af3714dc6babe46af31ae7a5c', 'Header 2': "Let's get started"}),
  Document(page_content='Download Fusion 360 for personal, hobby use.', metadata={'format': 'html', 'id': 'adsk-3d37d5a102de4f6ab983ad1075a9de0a', 'Header 2': "Let's get started"}),
  Document(page_content='Download Fusion 360 for personal, hobby use.', metadata={'format': 'html', 'id': 'adsk-dfcee855d4004f

In [523]:
demo_ephemeral_chat_history.add_user_message("How can it be used for product design?")
demo_ephemeral_chat_history

ChatMessageHistory(messages=[HumanMessage(content='What does Fusion 360 do?'), AIMessage(content='Fusion 360 is a cloud-based 3D modeling, CAD, CAM, CAE, and PCB software platform for product design and manufacturing.'), HumanMessage(content='How can it be used for product design?')])

In [524]:
query_transformation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are given a chat between an AI and a Human. Convert the last round of human input to a complete and self-contained question based on the whole conversation."
            "The generated self-contained question should be a stand-alone question that can be used as input to an information retrieval tool."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [531]:
from langchain_core.messages import BaseMessage

In [568]:
def parse_message_to_query(message: BaseMessage):
    return message.content

In [563]:
stand_alone_query_gen_chain = query_transformation_prompt | chat | parse_message_to_query

In [576]:
stand_alone_query_gen_chain = RunnablePassthrough.assign(query=query_transformation_prompt | chat | parse_message_to_query)

In [577]:
resp = stand_alone_query_gen_chain.invoke({
    "messages": demo_ephemeral_chat_history.messages,
    "chunk_num": 15
})

resp

{'messages': [HumanMessage(content='What does Fusion 360 do?'),
  AIMessage(content='Fusion 360 is a cloud-based 3D modeling, CAD, CAM, CAE, and PCB software platform for product design and manufacturing.'),
  HumanMessage(content='How can it be used for product design?')],
 'chunk_num': 15,
 'query': 'How can Fusion 360 be used for product design and manufacturing?'}

In [600]:
# ret_chain = stand_alone_query_gen_chain | {
#     "query": RunnablePassthrough.assign(),
#     "context": RunnableLambda(invoke_vector)
# }

# ret_chain = RunnablePassthrough(stand_alone_query_gen_chain) | RunnableLambda(invoke_vector)

ret_chain = stand_alone_query_gen_chain.assign(context=RunnableLambda(invoke_vector))
rag_chain = ret_chain.assign(answer=document_chain)

In [604]:
final_resp = rag_chain.invoke({
    "messages": demo_ephemeral_chat_history.messages,
    "chunk_num": 10
})

final_resp['answer']

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 61.03it/s]


'Fusion 360 can be used for product design by providing access to 3D design and modeling tools that are manufacturing-aware and automate design processes, managing design changes at any stage of production using pre-built workflows, and investigating the performance and manufacturability of designs with integrated simulation.'

In [534]:
retrieval_chain = RunnablePassthrough.assign(
    context=RunnablePassthrough.assign(query=stand_alone_query_gen_chain) | RunnableLambda(invoke_vector),
).assign(
    answer=document_chain,
)

In [538]:
resp = rag_chain.invoke({
    "messages": demo_ephemeral_chat_history.messages,
    "chunk_num": 15
})

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 55.43it/s]


In [540]:
resp.keys()

dict_keys(['messages', 'chunk_num', 'context', 'answer'])

In [597]:
test_rag_chain = (
    {"context": RunnableLambda(lambda x: x.lower()), "question": RunnablePassthrough()} | RunnableLambda(lambda x: x['question'] + " akbar")
)

In [598]:
test_rag_chain.invoke("Hi how are you?")

'Hi how are you? akbar'

In [603]:
lg_tracing_api_key = "ls__8e574352d1bf4f2a94ebfbc85a1cf23a"
os.environ['LANGCHAIN_TRACING_V2'] = "false"
os.environ['LANGCHAIN_API_KEY'] = lg_tracing_api_key

In [606]:
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import Tool

In [620]:
ret_tool = Tool(
    name="Information Retrieval",
    func=vector.similarity_search,
    description="A tool for obtaining infromation from external sources. You should use this tool to answer user questions correctly.",
    args_schema=RetToolInput,
)

In [633]:
retriever = vector.as_retriever(search_kwargs={'k': 6})
len(retriever.invoke("what is autodesk?"))
ret_tool = create_retriever_tool(
    retriever=retriever,
    name="Information Retrieval",
    description="A tool for obtaining infromation from external sources. You should use this tool to answer user questions correctly."
)

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 34.88it/s]


In [619]:
from langchain_core.pydantic_v1 import BaseModel
class RetToolInput(BaseModel):
    query: str
    k: int


In [626]:
inp = RetToolInput(**{"query": "what is autodesk?", "k": 10})

In [638]:
print(ret_tool.run("what is autodesk?"))

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 33.75it/s]

What is Autodesk Platform Services?

What is Autodesk Platform Services?

Autodesk is a leader in 3D design, engineering and entertainment software.  
About us Careers Contact us Investor relations Trust center Newsroom Suppliers Affiliate program

Autodesk is a leader in 3D design, engineering and entertainment software.  
About us Careers Contact us Investor relations Trust center Newsroom Suppliers Affiliate program

Autodesk
Autodesk is a leader in 3D design, engineering and entertainment software.
About us Careers Contact us Investor relations Trust center Newsroom Suppliers Affiliate Program Referral Program
//value = template=/etc/designs/autodesk/adsk-design/images/VideoPlayer_var.swf&source= //flashvars = template=/etc/designs/autodesk/adsk-design/images/VideoPlayer_var.swf&source=
of
Privacy settings | Privacy/Cookies | About our Ads | Legal | Report Noncompliance | Do not sell my personal information | Site map | ©️ 2020 Autodesk Inc. All rights reserved

Autodesk
Autodesk i

In [660]:
from langchain.agents import create_openai_functions_agent
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. You may not need to use the Information Retrieval tool for every query - the user may just want to chat!",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
print(prompt.messages)

agent = create_openai_functions_agent(chat, [ret_tool], prompt)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant. You may not need to use the Information Retrieval tool for every query - the user may just want to chat!')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')]


In [665]:
demo_ephemeral_chat_history.messages

[HumanMessage(content='What does Fusion 360 do?'),
 AIMessage(content='Fusion 360 is a cloud-based 3D modeling, CAD, CAM, CAE, and PCB software platform for product design and manufacturing.'),
 HumanMessage(content='How can it be used for product design?')]

In [662]:
agent_executor = AgentExecutor(agent=agent, tools=[ret_tool], verbose=True)

In [663]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are 

In [ ]:
from langchain.agents import AgentExecutor

In [ ]:
# 2. Create Tools
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
search = TavilySearchResults()
tools = [retriever_tool, search]


# 3. Create Agent
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

# We need to add these input/output schemas because the current AgentExecutor
# is lacking in schemas.

class Input(BaseModel):
    input: str
    chat_history: List[BaseMessage] = Field(
        ...,
        extra={"widget": {"type": "chat", "input": "location"}},
    )


class Output(BaseModel):
    output: str

add_routes(
    app,
    agent_executor.with_types(input_type=Input, output_type=Output),
    path="/agent",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)